In [1]:
import pandas as pd
import scanpy as sc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import os
import re


In [2]:
out_dir = "output"
os.makedirs(out_dir, exist_ok=True)

## Reading in the clinical data for patients

In [3]:
df = pd.read_excel('./INPUT/Daten_HHGNRegistry_JE_UKEbox_250430.xlsx')
df

,P-Number,Xenium Nr.,Alter (Jahre),Geschlecht,Fall,Time Bx-Lab (days),Albuminurie g/g,Serum-kreatinin,eGFR,C3,...,ds DNA,Treatmen7 days prior to biopsy,Sl_num,Disease,Pathology_num,GNregistry_num,ARRscore_SLEclass,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,NaN,X6,81,männlich,N00828/16,na,0.01,1.2,55,NaN,...,---,Steroid (75mg oral),1,ANCA,N00828/16,16_0828,ARRS low (0/11),NaN,NaN,low: 7
1,NaN,X3,55,weiblich,N03398/16,5,0.44,1.6,35,,...,,Steroid (2x70mg oral),2,ANCA,N03398/16,16_3398,ARRS medium (2/11),NaN,NaN,med: 16
2,NaN,X4,59,männlich,N03552/16,7,1.2,2.4,28.26,NaN,...,NaN,naive,3,ANCA,N03552/16,16_3552,ARRS medium (2/11),NaN,NaN,high: 6
3,NaN,X17,72,weiblich,N00194/17,-2,0.425,1.1,49.93,NaN,...,NaN,Steroid (3x unklare Dosis),4,ANCA,N00194/17,17_0194,ARRS low (0/11),NaN,NaN,NaN
4,NaN,X25,52,männlich,N00554/17,-1,3.25,3.7,17.59,NaN,...,NaN,Steroid (2x500),5,ANCA,N00554/17,17_0554,ARRS high (8/11),NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,NaN,NaN,31,weiblich,R02511/22,0,0.386,0.78,101.620003,13,...,187,NaN,24,SLE-Reserve,R02511/22,22_2511,NaN,NaN,NaN,NaN
73,NaN,X55,38,weiblich,R02672/18,na,na,na,na,na,...,NaN,na,22,SLE-Reserve,R02672/18,18_2672,Klasse IV+V,NaN,NaN,NaN
74,NaN,NaN,39,weiblich,R02961/22,-3,0.034,0.81,91.139999,70,...,206,NaN,25,SLE-Reserve,R02961/22,22_2961,NaN,NaN,NaN,NaN
75,NaN,NaN,42,weiblich,R03259/22,1,7.571,0.98,71.379997,NaN,...,NaN,NaN,26,SLE-Reserve,R03259/22,22_3259,NaN,NaN,NaN,NaN


In [4]:
df = df.iloc[:,:-3] # removing the last 3 additinal empty columns

In [5]:
df = df[df['Xenium Nr.'].notna()]
df

,P-Number,Xenium Nr.,Alter (Jahre),Geschlecht,Fall,Time Bx-Lab (days),Albuminurie g/g,Serum-kreatinin,eGFR,C3,...,cANCA IFT,PR3 ELISA,GBM,ds DNA,Treatmen7 days prior to biopsy,Sl_num,Disease,Pathology_num,GNregistry_num,ARRscore_SLEclass
0,NaN,X6,81,männlich,N00828/16,na,0.01,1.2,55,NaN,...,---,127,---,---,Steroid (75mg oral),1,ANCA,N00828/16,16_0828,ARRS low (0/11)
1,NaN,X3,55,weiblich,N03398/16,5,0.44,1.6,35,,...,na,na,,,Steroid (2x70mg oral),2,ANCA,N03398/16,16_3398,ARRS medium (2/11)
2,NaN,X4,59,männlich,N03552/16,7,1.2,2.4,28.26,NaN,...,---,---,NaN,NaN,naive,3,ANCA,N03552/16,16_3552,ARRS medium (2/11)
3,NaN,X17,72,weiblich,N00194/17,-2,0.425,1.1,49.93,NaN,...,---,---,NaN,NaN,Steroid (3x unklare Dosis),4,ANCA,N00194/17,17_0194,ARRS low (0/11)
4,NaN,X25,52,männlich,N00554/17,-1,3.25,3.7,17.59,NaN,...,---,---,NaN,NaN,Steroid (2x500),5,ANCA,N00554/17,17_0554,ARRS high (8/11)
5,NaN,X27,46,männlich,N00669/17,0,0.76,4.7,13.74,NaN,...,640,---,NaN,NaN,Steroid (100mg p.o.),6,ANCA,N00669/17,17_0669,ARRS medium (5/11)
6,P001,X18,43,männlich,N00929/17,0,na,3.3,21.52,NaN,...,----,---,NaN,NaN,naive,7,ANCA,N00929/17,17_0929,ARRS medium (2/11)
7,NaN,X33,65,männlich,N01352/17,-2,na,na,na,NaN,...,na,na,NaN,NaN,na,8,ANCA,N01352/17,17_1352,na
8,NaN,X28,47,männlich,N01359/17,2,na,2.58,28.17,NaN,...,---,"6,7",NaN,"45,4",na,9,ANCA,N01359/17,17_1359,ARRS medium (2/11)
9,NaN,X50,24,weiblich,N01376/17,0,na,0.63,125.07,NaN,...,na,na,NaN,NaN,na,10,ANCA,N01376/17,17_1376,ARRS low (0/11)


## Subsetting columns from clinical data

In [7]:
#PatientData_plotting = df.drop(columns = ['P-Number','Fall','Time Bx-Lab (days)','C3', 'C4', 'pANCA IFT', 'pANCA ELISA', 'cANCA IFT', 'cANCA ELISA','GBM', 'ds DNA ', 'Treatmen7 days prior to biopsy', 'Sl_num','Pathology_num',
#       'GNregistry_num'])
#Was giving this error: KeyError: "['pANCA ELISA', 'cANCA ELISA'] not found in axis"

PatientData_plotting = df.drop(columns = ['P-Number','Fall','Time Bx-Lab (days)','C3', 'C4', 'pANCA IFT', 'MPO ELISA', 'cANCA IFT', 'PR3 ELISA','GBM', 'ds DNA ', 'Treatmen7 days prior to biopsy', 'Sl_num','Pathology_num',
       'GNregistry_num'])

PatientData_plotting['Xenium Nr.'] = PatientData_plotting['Xenium Nr.'].str.replace(r'\(.*?\)', '', regex=True) # removing the (?) from X54

In [8]:
x_list = [f'X{i}' for i in range(1, 64)]

missing_values = set(x_list) - set(PatientData_plotting['Xenium Nr.'])
missing_values

# 1,2,9,10,41,57 are controls. 

{'X1', 'X10', 'X2', 'X41', 'X57', 'X9'}

In [9]:
print(PatientData_plotting.eGFR.dtype)

object


## Clean up columns for plotting

In [10]:
columns_to_convert = ['Albuminurie g/g', 'Serum-kreatinin', 'eGFR']

for column in columns_to_convert:
    # Replace 'na' and other problematic strings with np.nan
    PatientData_plotting[column] = PatientData_plotting[column].replace(['na', 'NaN', None], np.nan)

    # Convert column to numeric, coercing errors into NaN
    PatientData_plotting[column] = pd.to_numeric(PatientData_plotting[column], errors='coerce')

    # Optional: Convert to nullable integer type if needed
    #PatientData_plotting[column] = PatientData_plotting[column].astype('Int64')  # If you want integers with NaNs allowed


# Check updated DataFrame
print(PatientData_plotting.info())


<class 'pandas.core.frame.DataFrame'>
Index: 57 entries, 0 to 73
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Xenium Nr.         57 non-null     object 
 1   Alter (Jahre)      57 non-null     int64  
 2   Geschlecht         57 non-null     object 
 3   Albuminurie g/g    46 non-null     float64
 4   Serum-kreatinin    51 non-null     float64
 5   eGFR               51 non-null     float64
 6   Disease            57 non-null     object 
 7   ARRscore_SLEclass  57 non-null     object 
dtypes: float64(3), int64(1), object(4)
memory usage: 4.0+ KB
None


/tmp/ipykernel_11045/2266689106.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  PatientData_plotting[column] = PatientData_plotting[column].replace(['na', 'NaN', None], np.nan)


In [11]:
print(PatientData_plotting.eGFR.dtype)

float64


In [12]:
PatientData_plotting

,Xenium Nr.,Alter (Jahre),Geschlecht,Albuminurie g/g,Serum-kreatinin,eGFR,Disease,ARRscore_SLEclass
0,X6,81,männlich,0.010,1.20,55.000000,ANCA,ARRS low (0/11)
1,X3,55,weiblich,0.440,1.60,35.000000,ANCA,ARRS medium (2/11)
2,X4,59,männlich,1.200,2.40,28.260000,ANCA,ARRS medium (2/11)
3,X17,72,weiblich,0.425,1.10,49.930000,ANCA,ARRS low (0/11)
4,X25,52,männlich,3.250,3.70,17.590000,ANCA,ARRS high (8/11)
5,X27,46,männlich,0.760,4.70,13.740000,ANCA,ARRS medium (5/11)
6,X18,43,männlich,NaN,3.30,21.520000,ANCA,ARRS medium (2/11)
7,X33,65,männlich,NaN,NaN,NaN,ANCA,na
8,X28,47,männlich,NaN,2.58,28.170000,ANCA,ARRS medium (2/11)
9,X50,24,weiblich,NaN,0.63,125.070000,ANCA,ARRS low (0/11)


In [13]:
na_rows=PatientData_plotting[PatientData_plotting.isna().any(axis=1)]
na_rows
#Can we get these values?

,Xenium Nr.,Alter (Jahre),Geschlecht,Albuminurie g/g,Serum-kreatinin,eGFR,Disease,ARRscore_SLEclass
6,X18,43,männlich,NaN,3.30,21.52,ANCA,ARRS medium (2/11)
7,X33,65,männlich,NaN,NaN,NaN,ANCA,na
8,X28,47,männlich,NaN,2.58,28.17,ANCA,ARRS medium (2/11)
9,X50,24,weiblich,NaN,0.63,125.07,ANCA,ARRS low (0/11)
25,X11,51,weiblich,NaN,3.60,13.80,ANCA,ARRS medium (5/11)
47,X21,66,männlich,NaN,NaN,NaN,GBM,>95%gloms affected
49,X29,59,männlich,NaN,4.21,14.00,GBM,"86,8% gloms affected"
52,X7,54,männlich,NaN,NaN,NaN,SLE,Klasse IIIa
58,X23,35,weiblich,NaN,NaN,NaN,SLE,Klasse III+V
63,X48,51,weiblich,NaN,NaN,NaN,SLE,


In [14]:
PatientData_plotting.columns

Index(['Xenium Nr.', 'Alter (Jahre)', 'Geschlecht', 'Albuminurie g/g',
       'Serum-kreatinin', 'eGFR', 'Disease', 'ARRscore_SLEclass'],
      dtype='object')

In [15]:
PatientData_plotting = PatientData_plotting.rename(columns={
    "Xenium Nr.": "biopsy_nr",
    "Alter (Jahre)": "age", 
    "Geschlecht": "sex",
    "Albuminurie g/g": "albuminuria_gg",
    "Serum-kreatinin": "serum_creatinine",
    "Disease": "disease",
    "ARRscore_SLEclass":"ARRscore_SLEclass"
    
})


In [16]:
PatientData_plotting.head()

,biopsy_nr,age,sex,albuminuria_gg,serum_creatinine,eGFR,disease,ARRscore_SLEclass
0,X6,81,männlich,0.010,1.2,55.00,ANCA,ARRS low (0/11)
1,X3,55,weiblich,0.440,1.6,35.00,ANCA,ARRS medium (2/11)
2,X4,59,männlich,1.200,2.4,28.26,ANCA,ARRS medium (2/11)
3,X17,72,weiblich,0.425,1.1,49.93,ANCA,ARRS low (0/11)
4,X25,52,männlich,3.250,3.7,17.59,ANCA,ARRS high (8/11)


In [17]:
PatientData_plotting.to_csv('INPUT/PatientData_plotting.csv', index=False)

In [18]:
## Clinical Data for controls

In [19]:
ControlsData=pd.read_excel('./INPUT/Clinical_Data_HC_JE_Jonas_email_250417.xlsx')

In [20]:
ControlsData

,Xeniums Slide ID,Histo ID,CN-Nr.,Age,Sex,Creatinine [mg/dl],GFR [ml/min],Disease,Unnamed: 8,Unnamed: 9
0,X1,PA166/23,CN38,54,female,0.57,106,Liposarcoma retroperitoneal,NaN,NaN
1,X2,PA175/23,CN39,71,female,0.93,61,Liposarcoma retroperitoneal,NaN,same patient
2,X9,PA172/23,CN39,71,female,0.93,61,Liposarcoma retroperitoneal,NaN,NaN
3,X10,PA84/23,CN36,61,female,0.72,90,Transitional cell carcinoma,NaN,NaN
4,X41,PA38/22,CN27,57,male,1.07,91,clear cell renal cell carcinoma,NaN,NaN
5,X57,PA85/21,CN18,78,male,1.20,63,Transitional cell carcinoma,NaN,NaN


In [21]:
ControlsData= ControlsData.iloc[:,:-2]

In [22]:
ControlsData = ControlsData.rename(columns={
    "Xeniums Slide ID": "biopsy_nr",
    "Age": "age", 
    "Sex": "sex",
    "Creatinine [mg/dl]": "serum_creatinine_mg_dl",
    "GFR [ml/min]":"eGFR_ml_min",
    "Disease": "disease"
})


In [23]:
ControlsData_plotting= ControlsData.drop(columns = ["Histo ID","CN-Nr."])

In [24]:
ControlsData_plotting

,biopsy_nr,age,sex,serum_creatinine_mg_dl,eGFR_ml_min,disease
0,X1,54,female,0.57,106,Liposarcoma retroperitoneal
1,X2,71,female,0.93,61,Liposarcoma retroperitoneal
2,X9,71,female,0.93,61,Liposarcoma retroperitoneal
3,X10,61,female,0.72,90,Transitional cell carcinoma
4,X41,57,male,1.07,91,clear cell renal cell carcinoma
5,X57,78,male,1.20,63,Transitional cell carcinoma


In [25]:
ControlsData_plotting.to_csv('./INPUT/ControlsData_plotting.csv', index=False)

## Putting the control and pateint clinical data together

In [26]:
import pandas as pd

def append_rows_with_matching_columns(df_base, df_to_append, column_map):
    """
    Rename and align df_to_append to match df_base, filling missing columns with NaN.
    
    Parameters:
        df_base (pd.DataFrame): The reference DataFrame with desired column structure.
        df_to_append (pd.DataFrame): The new rows to add (columns may differ).
        column_map (dict): Mapping from df_to_append columns → df_base columns.

    Returns:
        pd.DataFrame: Combined DataFrame with consistent columns.
    """
    # Step 1: Rename df_to_append columns to match df_base
    df_renamed = df_to_append.rename(columns=column_map)

    # Step 2: Add missing columns (fill with NA)
    for col in df_base.columns:
        if col not in df_renamed.columns:
            df_renamed[col] = pd.NA

    # Step 3: Reorder to match df_base
    df_aligned = df_renamed[df_base.columns]

    # Step 4: Concatenate
    combined_df = pd.concat([df_base, df_aligned], ignore_index=True)

    return combined_df


In [27]:
column_names_matched = {
    "serum_creatinine_mg_dl": "serum_creatinine",
    "eGFR_ml_min": "eGFR"
}

clinical_data_all = append_rows_with_matching_columns(PatientData_plotting,
                                             ControlsData_plotting,
                                             column_map = column_names_matched)


/tmp/ipykernel_11045/2209758065.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([df_base, df_aligned], ignore_index=True)


In [28]:
clinical_data_all

,biopsy_nr,age,sex,albuminuria_gg,serum_creatinine,eGFR,disease,ARRscore_SLEclass
0,X6,81,männlich,0.010,1.20,55.00,ANCA,ARRS low (0/11)
1,X3,55,weiblich,0.440,1.60,35.00,ANCA,ARRS medium (2/11)
2,X4,59,männlich,1.200,2.40,28.26,ANCA,ARRS medium (2/11)
3,X17,72,weiblich,0.425,1.10,49.93,ANCA,ARRS low (0/11)
4,X25,52,männlich,3.250,3.70,17.59,ANCA,ARRS high (8/11)
...,...,...,...,...,...,...,...,...
58,X2,71,female,NaN,0.93,61.00,Liposarcoma retroperitoneal,NaN
59,X9,71,female,NaN,0.93,61.00,Liposarcoma retroperitoneal,NaN
60,X10,61,female,NaN,0.72,90.00,Transitional cell carcinoma,NaN
61,X41,57,male,NaN,1.07,91.00,clear cell renal cell carcinoma,NaN


In [29]:
clinical_data_all.to_csv(os.path.join(out_dir,"clinical_data_all.csv"), index=False)